In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os

# === 1. Ler base de dados (shapefile) ===
SHP_PATH = r"..\..\Data\Processed\PT-FireSprd_v2.1\L2_FireBehavior\PT-FireProg_v2.1_L2_final.shp"
gdf = gpd.read_file(SHP_PATH)

# Converter para pandas DataFrame (mantendo apenas atributos)
df = pd.DataFrame(gdf.drop(columns=gdf.geometry.name, errors="ignore"))

# === 2. Selecionar apenas variáveis numéricas ===
df_num = df.select_dtypes(include=[np.number]).copy()

# === 3. Detectar automaticamente a coluna alvo 'ros p' ===
target_candidates = [c for c in df_num.columns if "ros" in c.lower() and "p" in c.lower()]
if target_candidates:
    target_col = target_candidates[0]
else:
    raise ValueError("❌ Não encontrei nenhuma coluna parecida com 'ros p' na base de dados.")

print(f"✅ Coluna alvo identificada: {target_col}")

# === 4. Função de transformação SEGURA (mantém valores válidos) ===
def transformar(series, tipo):
    series = pd.to_numeric(series, errors="coerce")
    series = series.fillna(series.median())  # substitui NA por mediana
    if tipo == "linear":
        return series
    elif tipo == "log":
        # desloca toda a série para valores positivos antes de aplicar log
        minv = series.min()
        if minv <= 0:
            series = series - minv + 1e-6
        return np.log1p(series)
    elif tipo == "sqrt":
        # garante não-negatividade
        minv = series.min()
        if minv < 0:
            series = series - minv
        return np.sqrt(series)
    else:
        raise ValueError("Transformação desconhecida.")

# === 5. Combinações pedidas ===
combinacoes = [
    ("linear", "linear"),
    ("linear", "log"),
    ("log", "log"),
    ("log", "linear"),
    ("linear", "sqrt"),
    ("sqrt", "sqrt"),
    ("sqrt", "linear"),
]

# === 6. Pasta de saída ===
os.makedirs("datasets_transformados_validados", exist_ok=True)

# === 7. Loop principal ===
for t_ros, t_outras in combinacoes:
    print(f"\n➡ Transformação: ros p ({t_ros}) vs variáveis ({t_outras})")

    # Transformar ros p
    ros_t = transformar(df_num[target_col], t_ros)
    df_transf = pd.DataFrame({target_col: ros_t})

    # Transformar restantes variáveis
    for col in df_num.columns:
        if col == target_col:
            continue
        df_transf[col] = transformar(df_num[col], t_outras)

    # Garantir que não há colunas totalmente vazias
    df_transf = df_transf.fillna(df_transf.median())

    # Calcular correlações
    corr = df_transf.corr()[target_col].sort_values(ascending=False)

    # Guardar dataset transformado
    out_name = f"datasets_transformados_validados/{t_ros}_{t_outras}.csv"
    df_transf.to_csv(out_name, index=False, sep=";")

    print(f"  → Guardado: {out_name}")
    print(f"  → Linhas: {len(df_transf)}, Colunas: {len(df_transf.columns)}")
    print("  Correlações mais fortes:")
    print(corr.head(10))
    print("  ---")

print("\n✅ Todos os datasets transformados foram gerados em 'datasets_transformados_validados/'")

c:\Users\dias3\miniconda3\envs\geoenv\Lib\site-packages\pyogrio\core.py:35: RuntimeWarning: Could not detect GDAL data files.  Set GDAL_DATA environment variable to the correct path.
  _init_gdal_data()


✅ Coluna alvo identificada: ros_p

➡ Transformação: ros p (linear) vs variáveis (linear)


C:\Users\dias3\AppData\Local\Temp\ipykernel_8068\2369730359.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_transf[col] = transformar(df_num[col], t_outras)
C:\Users\dias3\AppData\Local\Temp\ipykernel_8068\2369730359.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_transf[col] = transformar(df_num[col], t_outras)
C:\Users\dias3\AppData\Local\Temp\ipykernel_8068\2369730359.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

  → Guardado: datasets_transformados_validados/linear_linear.csv
  → Linhas: 3355, Colunas: 121
  Correlações mais fortes:
ros_p         1.000000
int_p         0.970662
ros_i         0.863803
int_i         0.851008
growth_rat    0.785923
inidoy        0.510580
ros_p_lg1     0.508118
area          0.438615
wSv_9_av      0.401141
gT_s_9_av     0.399842
Name: ros_p, dtype: float64
  ---

➡ Transformação: ros p (linear) vs variáveis (log)


C:\Users\dias3\AppData\Local\Temp\ipykernel_8068\2369730359.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_transf[col] = transformar(df_num[col], t_outras)
C:\Users\dias3\AppData\Local\Temp\ipykernel_8068\2369730359.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_transf[col] = transformar(df_num[col], t_outras)
C:\Users\dias3\AppData\Local\Temp\ipykernel_8068\2369730359.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

...

  → Guardado: datasets_transformados_validados/linear_sqrt.csv
  → Linhas: 3355, Colunas: 121
  Correlações mais fortes:
ros_p         1.000000
int_p         0.890041
growth_rat    0.839683
ros_i         0.807801
int_i         0.799090
area          0.562550
inidoy        0.492688
ros_p_lg1     0.465536
wSv_9_av      0.400622
gT_s_9_av     0.399909
Name: ros_p, dtype: float64
  ---

➡ Transformação: ros p (sqrt) vs variáveis (sqrt)


C:\Users\dias3\AppData\Local\Temp\ipykernel_8068\2369730359.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_transf[col] = transformar(df_num[col], t_outras)
C:\Users\dias3\AppData\Local\Temp\ipykernel_8068\2369730359.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_transf[col] = transformar(df_num[col], t_outras)
C:\Users\dias3\AppData\Local\Temp\ipykernel_8068\2369730359.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

  → Guardado: datasets_transformados_validados/sqrt_sqrt.csv
  → Linhas: 3355, Colunas: 121
  Correlações mais fortes:
ros_p         1.000000
int_p         0.987275
ros_i         0.930247
int_i         0.923053
growth_rat    0.876526
inidoy        0.771116
area          0.627390
spdir_p       0.620215
spdir_i       0.587957
duration_i    0.580241
Name: ros_p, dtype: float64
  ---

➡ Transformação: ros p (sqrt) vs variáveis (linear)


C:\Users\dias3\AppData\Local\Temp\ipykernel_8068\2369730359.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_transf[col] = transformar(df_num[col], t_outras)
C:\Users\dias3\AppData\Local\Temp\ipykernel_8068\2369730359.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_transf[col] = transformar(df_num[col], t_outras)
C:\Users\dias3\AppData\Local\Temp\ipykernel_8068\2369730359.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

  → Guardado: datasets_transformados_validados/sqrt_linear.csv
  → Linhas: 3355, Colunas: 121
  Correlações mais fortes:
ros_p         1.000000
int_p         0.881895
ros_i         0.826756
int_i         0.819564
inidoy        0.775274
growth_rat    0.673704
fire_rank     0.521775
spdir_p       0.510937
wSv_9_av      0.508516
gT_s_9_av     0.507842
Name: ros_p, dtype: float64
  ---

✅ Todos os datasets transformados foram gerados em 'datasets_transformados_validados/'
